In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import caiman as cm
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
import numpy as np
from copy import deepcopy
import os.path
import json
from nd2_to_caiman import np_arr_from_nd2
import scipy
from RippleNoiseRemoval import RNR
from time import time
from movie_splitting import numpy_to_hdf5
from numba import njit, prange
import h5py

In [ ]:
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from copy import deepcopy

## Open CNMF file

In [ ]:
cnmf_fpath = fh.open_file("Open hdf5 caiman file")
print(cnmf_fpath)

In [ ]:
cnmf = cm.source_extraction.cnmf.cnmf.load_CNMF(cnmf_fpath)

In [ ]:
session_uuid = None
with h5py.File(cnmf_fpath, 'r') as hf:
    session_uuid = hf.attrs["uuid"]
print(f"UUID of session was {session_uuid}")

## Open MoCo parameters and other parameters

In [ ]:
# assumed naming conventions:
# CNMF results: xy_cnmf.hdf5
# moco parameters: xy_moco_pars.h5
# CNMF parameters: xy_pars.json

# get root file name (and path)
# get rid of extension and "_cnmf" at the end
root_fpath = "_".join(os.path.splitext(cnmf_fpath)[0].split("_")[:-1])


pars_fpath = root_fpath + "_pars.json"
moco_pars_fpath = root_fpath + "_moco_pars.h5"

In [ ]:
with open(pars_fpath, "r") as json_file:
    js = json.load(json_file)

In [ ]:
nd2_fpath = js["original_fnames"]
print(nd2_fpath)
print(f"nd2 file available: {os.path.exists(nd2_fpath)}")

In [ ]:
moco_pars = dict()
moco_data_dict = dict()
def bytesListToList(blist):
    return list(map(lambda x: None if x == 'None' else x, blist.decode("utf-8") .rstrip()[1:-1].split(", ")))
def bytesToVal(bs):
    return None if bs == b'None' else bs.decode('utf-8') # TODO: convert to double?

with h5py.File(moco_pars_fpath, "r") as hf:
    for key in hf.keys():
        moco_data_dict[key] = hf[key][()]

    moco_pars["border_nan"] = hf.attrs["border_nan"]
    moco_pars['uuid'] = hf.attrs["uuid"]
    moco_pars["var_name_hdf5"] = hf.attrs["var_name_hdf5"].decode("utf-8")
    
    
    

In [ ]:
moco_data_dict.keys()

## Set output file

In [ ]:
# set export folder for temporary files
export_folder = fh.open_dir("Select folder to save results", True)
# export_fname: get rid of .nd2 extension, append date and .h5 extension
export_fname = fh.get_filename_with_date(os.path.splitext(os.path.split(nd2_fpath)[1])[0] + "_rnr_", ".hdf5")
export_hd5_fpath = os.path.join(export_folder, export_fname) # nd2_fpath.split("/")[-1][:-4] + "_exp.h5"
print(f"Export file selected: {export_hd5_fpath}")

In [ ]:
if not("export_hd5_fpath" in locals()):
    export_hd5_fpath = fh.open_file("Choose hd5 file to open")
if export_hd5_fpath.split(".")[-1] != "hdf5":
    print(f"Invalid hd5 file:\n{export_hd5_fpath}\nChoose a valid hd5 file!")
    export_hd5_fpath = fh.open_file("Choose hd5 file to open")
fnames = [export_hd5_fpath]
print(f"Going to perform MoCo on {fnames}")
assert export_hd5_fpath.split(".")[-1] == "hdf5", f"Invalid file extension: .{export_hd5_fpath.split('.')[-1]}, expected .hdf5"

# Extract spatial and temporal components from CNMF object

In [ ]:
temporal = cnmf.estimates.C  # FIXME: this is not the raw signal!
spatial = cnmf.estimates.A.todense()

In [ ]:
print(f"spatial: {spatial.shape}\ntemporal: {temporal.shape}")

### Get segmentation values
First, needed for checking segmentation consistency.

In [ ]:
# Old version of saving intervals for MoCo and CNMF:
if "moco_intervals" in js.keys():
    moco_intervals = js["moco_intervals"]
    moco_flags = js["moco_flags"]
    cnmf_intervals = js["cnmf_intervals"]
    cnmf_flags = js["cnmf_flags"]
elif "moco_intervals" in moco_data_dict.keys():
    moco_intervals = moco_data_dict["moco_intervals"]
    moco_flags = moco_data_dict["moco_flags"]
    cnmf_intervals = moco_data_dict["cnmf_intervals"]
    cnmf_flags = moco_data_dict["cnmf_flags"]
else:
    raise Exception("No segmentation info found.")

# Perform RNR
* TODO: it is not necessary to perform, as the difference is tiny to non-existent in the traces.

In [ ]:
win = js["rnr_win"]
amplitude_threshold = js["amplitude_threshold"]

In [ ]:
rnr = RNR(win, amplitude_threshold) 

In [ ]:
t0_open = time()
if "begin_end_frames" in locals():
    rnr.open_recording(nd2_fpath, begin_end_frames)  # opens usual recording size (8.8-9 GB) in about 830 s
else:
    rnr.open_recording(nd2_fpath)
print(f"File opened in {time() - t0_open} s")

### Test consistency of segmentation

In [ ]:
for i in range(1, len(cnmf_intervals)):
    gap = cnmf_intervals[i][0] - cnmf_intervals[i-1][1]
    if gap != 1:
        raise Exception(f"Error in segmentation: Intervals {i} and {i+1}: {cnmf_intervals[i-1]} {cnmf_intervals[i]} are not continuous! Difference of {gap}")
if cnmf_intervals[0][0] != 1:
    raise Exception(f"Error in segmentation: does not start with frame 1: {cnmf_intervals[0]}")
if cnmf_intervals[-1][1] != rnr.nd2_data.shape[0]:
    raise Exception(f"Error in segmentation: does not seem to cover whole recording! Last segment: {cnmf_intervals[-1]}, length of movie: { rnr.nd2_data.shape[0]}")

In [ ]:
for i in range(1, len(cnmf_intervals)):
    gap = cnmf_intervals[i][0] - cnmf_intervals[i-1][1]
    print(f"{cnmf_intervals[i]}\t{gap}")

In [ ]:
multi_core = True
if multi_core:
    import multiprocessing as mp
    n_processes = mp.cpu_count() - 2  # leave some capacity for possible other notebooks etc.
    t0_multi = time()
    rnr_data = rnr.rnr_multithread(n_processes)  # a bit faster than opening file, around 500s for 8.8-9 GB
    t1_multi = time()
    print(f"RNR multi-thread with {n_processes} processes finished in {t1_multi - t0_multi} s")
else:
    t0_single = time()
    rnr_data = rnr.rnr_singlethread()  # a bit faster than opening file, around 500s for 8.8-9 GB
    t1_single = time()
    print(f"RNR single thread finished in {t1_single - t0_single} s")
print(f"Result is a {type(rnr_data)} with datatype {rnr_data.dtype}")
print(f"Shape: {rnr_data.shape[0]} frames of {rnr_data.shape[1]}x{rnr_data.shape[2]} pixels")

## Save results

In [ ]:
numpy_to_hdf5(rnr_data, export_hd5_fpath)

## (Optional) Save no-rnr results to same type of hdf5 for comparison

In [ ]:
compare_nornr = False
if compare_nornr:
    export_hd5_fpath_nornr = os.path.splitext(export_hd5_fpath)[0] + "_nornr.hdf5"
    numpy_to_hdf5(rnr.nd2_data, export_hd5_fpath_nornr)

## Clean up memory

In [ ]:
del rnr
del rnr_data

# MoCo

### Get other parameters (CNMF and moco)

In [ ]:
# dataset dependent parameters
fr = js["fr"]                             # imaging rate in frames per second
decay_time = js["decay_time"]                  # length of a typical transient in seconds

# motion correction parameters
strides = tuple(js["strides"])          # start a new patch for pw-rigid motion correction every x pixels
overlaps = tuple(js["overlaps"])         # overlap between pathes (size of patch strides+overlaps)
max_shifts = tuple(js["max_shifts"])          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = js["max_deviation_rigid"]     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = js["pw_rigid"]             # flag for performing non-rigid motion correction

# parameters for source extraction and deconvolution
p = js["p"]                       # order of the autoregressive system
gnb = js["nb"]                     # number of global background components
merge_thr = js["merge_thr"]            # merging threshold, max correlation allowed
rf = js["rf"]                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = js["stride"]             # amount of overlap between the patches in pixels
K = js["K"]                       # number of components per patch
gSig = js["gSig"]               # expected half size of neurons in pixels
method_init = js["method_init"]  # initialization method (if analyzing dendritic data using 'sparse_nmf')
ssub = js["ssub"]                    # spatial subsampling during initialization
tsub = js["tsub"]                    # temporal subsampling during intialization

# parameters for component evaluation
min_SNR = js["min_SNR"]               # signal to noise ratio for accepting a component
rval_thr = js["rval_thr"]              # space correlation threshold for accepting a component
cnn_thr = js["min_cnn_thr"]              # threshold for CNN based classifier
cnn_lowest = js["cnn_lowest"] # neurons with cnn probability lower than this value are rejected

In [ ]:
if "fnames" not in locals():
    fnames = fh.open_file("No hd5 file selected. Choose corresponding hd5 file!")
opts_dict = {'fnames': fnames, 
            'fr': fr,
            'decay_time': decay_time,
            'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid,
            'p': p,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub,
            'merge_thr': merge_thr, 
            'min_SNR': min_SNR,
            'rval_thr': rval_thr,
            'use_cnn': True,
            'min_cnn_thr': cnn_thr,
            'cnn_lowest': cnn_lowest,
            'var_name_hdf5': 'data',
            'gSig' :  gSig,}  # FIXME: does not work! Check where does this setting get lost?

opts = params.CNMFParams(params_dict=opts_dict)

### Set up cluster

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

### Apply shifts

In [ ]:
mc.x_shifts_els = moco_data_dict["x_shifts_els"]
mc.y_shifts_els = moco_data_dict["y_shifts_els"]

In [ ]:
work_folder = os.path.split(fnames[0])[0]
print(f"Changing work folder to {work_folder}, this is where moco result will be saved")

In [ ]:
os.chdir(work_folder)

In [ ]:
# should save in C order because cm.load_memmap() takes C-memmap. However, as the opening and closing of memmap files is 
# so confusing, I decided to try to copy the original demo_pipeline jupyter notebook as closely as I can.
exp_fname = mc.apply_shifts_movie(fnames, save_memmap=True,order="F")
print(exp_fname)

In [ ]:
mc_mmap = os.path.join(work_folder, exp_fname)

# Extract trace
`m_els` is the memmap that contains recording, RNR + MoCo (both optional).

In [ ]:
m_els = cm.load(mc_mmap)

In [ ]:
m_els.shape

### define dimensions

In [ ]:
n_neurons = temporal.shape[0]
n_frames_cut = temporal.shape[1]
n_frames_long = m_els.shape[0]
n_bkgd = cnmf.estimates.b.shape[1]  # number of background components. Usually 2 (or 1)

In [ ]:
print(f"CNMF contains temporal and spatial components of {n_neurons} neurons.\n\tTotal:\t{n_frames_long} frames\n\tCNMF:\t{n_frames_cut} frames\n\tDiff:\t{n_frames_long-n_frames_cut} frames\n(Diff: the frames that were cut out before running the CNMF algorithm (seizure etc.))")

In [ ]:
# check intervals are consistent with temporal size
n_frs = 0
for i_tup, tup in enumerate(cnmf_intervals):
    if cnmf_flags[i_tup]:
        n_frs += tup[1] - tup[0] + 1
print(n_frs)
print(n_frs == n_frames_cut)

### create np arrays for whole traces and traces of only segments given to CNMF.

In [ ]:
neuron_weight = np.zeros(n_neurons, dtype=m_els.dtype)
for i_neuron in range(n_neurons):
    weight = 0
    neuron_mask = spatial[:,i_neuron].reshape((512,512)).transpose()  # do not confuse neuron mask with elon musk
    pixels_x, pixels_y = scipy.sparse.csc_matrix(neuron_mask).nonzero()
    for pix_x, pix_y in zip(pixels_x, pixels_y):
        weight += neuron_mask[pix_x, pix_y]
    neuron_weight[i_neuron] = weight

In [ ]:
moco_data = np.asarray(m_els.data)  # TODO: apparently, this does not load memmap into file, so might even skip this step...

### Reformat spatial matrix
WARNING: currently, resolution (512x512) is hard-coded, even though other resolutions might be used. This can be resolved by reading out the CNMF parameters, or the nikon movie parameters... Must pay attention to x and y where they are not symmetric.

In [ ]:
spatial = np.array(spatial)  # change type to allow np.reshape (?)
spatial = np.reshape(spatial, (512, 512, n_neurons)) # (262144 -> 512x512, i.e. "unflatten")
spatial = np.transpose(spatial, axes=[1,0,2])  # move neuron index to end

In [ ]:
print(f"Neuron components:\nSpatial dim: {spatial.shape}\nTemporal dim: {temporal.shape}")
assert spatial.shape[0] == 512
assert spatial.shape[1] == 512
assert spatial.shape[2] == n_neurons
assert temporal.shape[0] == n_neurons
assert temporal.shape[1] == n_frames_cut

In [ ]:
@njit(parallel=True)
def extract_neuron_trace():  # neuron_xy = np.array[x0, x1, ..., xn; y0, y1, ..., yn], 2 rows, n columns
    traces_par = np.zeros((n_neurons, n_frames_long), dtype=moco_data.dtype)
    for i_neuron in prange(n_neurons):
        pixels_x, pixels_y = spatial[:,:,i_neuron].nonzero()
        trace = np.zeros((n_frames_long,), dtype=moco_data.dtype)
        for i_frame in range(n_frames_long):
            trace_val = 0.
            spatial_bkgd = 0.
            i_pix = 0
            while i_pix < pixels_x.shape[0]:
                pix_neuron_value = spatial[pixels_x[i_pix], pixels_y[i_pix], i_neuron]*moco_data[i_frame, pixels_x[i_pix], pixels_y[i_pix]]
                trace_val = trace_val + pix_neuron_value
                i_pix = i_pix + 1
            traces_par[i_neuron, i_frame] = trace_val
        #np.sum(moco_data[:, pixels_x, pixels_y]*spatial[pixels_x, pixels_y, i_neuron], axis=1)
    return traces_par

In [ ]:
traces_complete = extract_neuron_trace()

In [ ]:
weights_par = np.sum(spatial, axis=(0,1))  # calculate total weight (sum of spatial values) per neuron

In [ ]:
# create normalized traces. Logic behind normalization is that the higher the spatial values, the more stronger the neuron is.
traces_complete_norm = np.divide(traces_complete, weights_par[:, None])

# Copy CNMF traces, fill missing segment values with manually extracted trace

## Extract CNMF "raw" trace (is it?) as seen in nb_view_components (blue)

In [ ]:
#cnmf.estimates.nb_view_components(denoised_color='red')

In [ ]:
img = np.reshape(np.array(cnmf.estimates.A.mean(axis=1)), cnmf.estimates.dims, order='F')

In [ ]:
Yr = cm.utils.visualization.nb_view_patches(
                None, cnmf.estimates.A, cnmf.estimates.C, cnmf.estimates.b, cnmf.estimates.f, cnmf.estimates.dims[0], cnmf.estimates.dims[1],
                YrA=cnmf.estimates.R, image_neurons=img, thr=0.99, denoised_color="red", cmap="jet",
                r_values=cnmf.estimates.r_values, SNR=cnmf.estimates.SNR_comp, cnn_preds=cnmf.estimates.cnn_preds)

## Project CNMF "raw" trace onto whole movie, fill difference with manual trace
The trace extracted by CNMF is shorter than the whole movie, and without the gaps (i.e. if CNMF was not performed on frames 10-20, then instead of 1-9, (10-20), 21-30, (31-40), 41-N as trace, we have 1-(N-21) consecutive frames. Using the cnmf intervals and corresponding flags, create a mapping between the original 1-N frames and the shortened version. I.e. to access the CNMF value for frame 21, using this mapping would give us the frame 10 (9 in 0-indexing).

In [ ]:
cnmf_traces = np.zeros(shape=(n_neurons, n_frames_long), dtype=cnmf.estimates.C.dtype)

In [ ]:
for i_segment in range(2, len(cnmf_intervals)):
    if not (cnmf_intervals[i_segment-1][1] == cnmf_intervals[i_segment][0] - 1):
        print(i_segment)
        print(cnmf_intervals[i_segment-1])
        print(cnmf_intervals[i_segment])
        print()

In [ ]:
# create adjusted intervals with index = i_segment, and value (i_beginning_whole, i_end_whole).

cnmf_intervals_adjusted = cnmf_intervals[cnmf_flags].copy()
i_segment_adjusted = 0
current_gap = 0

for i_segment, segment in enumerate(cnmf_intervals):
    if cnmf_flags[i_segment]:  # cnmf was performed on this segment
        # only adjust for the current indexing gap in the beginning and end frames
        cnmf_intervals_adjusted[i_segment_adjusted][0] -= current_gap
        cnmf_intervals_adjusted[i_segment_adjusted][1] -= current_gap
        print(f"{segment}\t{cnmf_intervals_adjusted[i_segment_adjusted]}\t{current_gap}\t{cnmf_flags[i_segment]}")
        i_segment_adjusted += 1
    else:  # cnmf was not performed; add segment length to current_gap
        seg_len = segment[1] - segment[0] + 1  # e.g. [1, 10] has length 10
        current_gap += seg_len
        print(f"{segment}\t\t\t{current_gap}\t{cnmf_flags[i_segment]}")
    

#if len(cnmf_intervals_adjusted) == 1:
#    diff = cnmf_intervals_adjusted[0][0] - 1
#    cnmf_intervals_adjusted[0][0] -= diff
#    cnmf_intervals_adjusted[0][1] -= diff
#else:
#    for i in range(1, len(cnmf_intervals_adjusted)):
#        segment_dist = cnmf_intervals_adjusted[i][1] - cnmf_intervals_adjusted[i][0]  # e.g. [1, 10] -> 1+9=10, so dist=10=10-1
#        cnmf_intervals_adjusted[i][0] = cnmf_intervals_adjusted[i-1][1] + 1
#        cnmf_intervals_adjusted[i][1] = cnmf_intervals_adjusted[i][0] + segment_dist

assert cnmf_intervals_adjusted[0][0] == 1

In [ ]:
temporal.shape

In [ ]:
adjusted_index = 0 
len_temporal = 0  # total length of cnmf=True segments; should match length of temporal
for i_seg, seg in enumerate(cnmf_intervals):
    if cnmf_flags[i_seg]:
        # test all matching segments have same length
        assert cnmf_intervals_adjusted[adjusted_index][1] - cnmf_intervals_adjusted[adjusted_index][0] == seg[1]-seg[0]
        len_temporal += seg[1] - seg[0] + 1
        adjusted_index += 1
assert len_temporal == temporal.shape[1]

In [ ]:
for i in range(1, len(cnmf_intervals_adjusted)):
    if not (cnmf_intervals_adjusted[i][0] - cnmf_intervals_adjusted[i-1][1]) == 1:
        print(i)

In [ ]:
cnmf_intervals[53:55]

In [ ]:
cnmf_intervals_adjusted[53:55]

In [ ]:
i_cnmf_segment = 0
# copy either cnmf trace or manually extracted trace to the segment.
for i_interval, interval in enumerate(cnmf_intervals):
    if cnmf_flags[i_interval]:  # cnmf was performed on this segment; just copy trace to right position
        for i_neuron in range(n_neurons):
            cnmf_traces[i_neuron, interval[0]-1 : interval[1]] = Yr[i_neuron, cnmf_intervals_adjusted[i_cnmf_segment][0]-1:cnmf_intervals_adjusted[i_cnmf_segment][1]]  # careful about 1-indexing
        i_cnmf_segment += 1
    else:  # need to fill segment with manual traces
        for i_neuron in range(n_neurons):
            cnmf_traces[i_neuron, interval[0]-1:interval[1]] = traces_complete[i_neuron, interval[0]-1:interval[1]]

# Shift baseline to match manual trace to cnmf signal

In [ ]:
for i_neuron in range(n_neurons):
    baseline_diff = np.median(traces_complete[i_neuron,:] - cnmf_traces[i_neuron,:])
    for i_interval, interval in enumerate(cnmf_intervals):
        if not cnmf_flags[i_interval]:
            for i_frame in range(interval[0]-1, interval[1]):  #interval 1-indexed, e.g. [1, 10]; convert to 0, ..., 9 (inclusive)
                cnmf_traces[i_neuron, i_frame] -= baseline_diff

# Plot results

### Compare cnmf raw signal with signal extracted from spatial x nd2 fluorescence averaging

In [ ]:
I_NEU = 40
fig = plt.figure(figsize=(18,10))
plt.plot(traces_complete[I_NEU], linewidth=0.5, color="blue")  # pure raw data
plt.plot(cnmf_traces[I_NEU], linewidth=0.5, color="black")  # cnmf intertwined with raw extracted segments
plt.plot(traces_complete[I_NEU]-cnmf_traces[I_NEU], color="yellow")
#plt.plot(cnmf_traces[20] + np.median(traces_complete[20] - cnmf_traces[20]), linewidth=3)
ax = plt.gca()
#ax.set_xlim((2000,3000))
plt.show()
# TODO: watch MoCo video to see (1359 T352) if something goes wrong there.

# Optional: compare with no-rnr data

In [ ]:
if compare_nornr:
    nornr_data = np_arr_from_nd2(nd2_fpath)
    numpy_to_hdf5(nornr_data, export_hd5_fpath_nornr)
    work_folder_nornr = # TODO: compare with no RNR!
    os.mkdir(work_folder_nornr)
    print(f"Changing to {work_folder_nornr}")
    os.chdir(work_folder_nornr)
    fnames_nornr = [export_hd5_fpath_nornr]
    mc_nornr = MotionCorrect(fnames_nornr, dview=dview, **opts.get_group('motion'))
    mc_nornr.x_shifts_els = moco_data_dict["x_shifts_els"]
    mc_nornr.y_shifts_els = moco_data_dict["y_shifts_els"]
    # should save in C order because cm.load_memmap() takes C-memmap. However, as the opening and closing of memmap files is 
    # so confusing, I decided to try to copy the original demo_pipeline jupyter notebook as closely as I can.
    exp_fname_nornr = mc.apply_shifts_movie(fnames_nornr, save_memmap=True,order="F")
    print(exp_fname_nornr)
    mc_mmap_nornr = os.path.join(work_folder_nornr, exp_fname_nornr)
    m_els_nornr = cm.load(mc_mmap_nornr)
    moco_data = np.asarray(m_els_nornr)
    traces_nornr = extract_neuron_trace()
    moco_data = np.asarray(m_els)
    fig = plt.figure(figsize=(18,10))
    plt.plot(traces_complete[20], linewidth=0.5, color="blue")  # pure raw data
    plt.plot(traces_nornr[20], linewidth=0.5, color="green")
    plt.plot(traces_complete[20]-traces_nornr[20], linewidth=0.5, color="yellow")
    #plt.plot(cnmf_traces[20] + np.median(traces_complete[20] - cnmf_traces[20]), linewidth=3)
    plt.show()
    rnr_diff = traces_complete - traces_nornr
    print(rnr_diff.max())

# TODO: use Nikon metadata to export time stamps in hdf5 file.

# Save traces and spatial masks to hdf5 file

In [ ]:
root_fname = os.path.split(moco_pars_fpath)[-1][:-13]

In [ ]:
# assuming the naming convention:
# moco_pars_fname = root_fpath + "_moco_pars.h5":
root_fname = os.path.split(moco_pars_fpath)[-1][:-13]
whole_traces_h5_fname = root_fname + "_traces.h5"
whole_traces_h5_fpath = os.path.join(export_folder, whole_traces_h5_fname)
print(f"Saving traces to\n{whole_traces_h5_fpath}")

In [ ]:
# taken from Pure Python Pipeline Splitting notebook
utf8_type = h5py.string_dtype('utf-8', 30)
def append_dataset(h5_file, name, data):
    if (type(data) is tuple and type(data[0]) is slice) \
    or \
    data is None \
    or \
    type(data) is str \
    or \
    (type(data) is list and (data[0] is None or type(data[0]) is str)):  
        # some entries (e.g. indices) are a tuple of slices
        # some entries are of type string, are None, [None, None, ...] or ["..."]
        # convert these types to string (easiest way to preserve information about format)
        #data_arr = np.array(, dtype=utf8_type)
        hf.attrs[name] = data.__str__().encode("utf-8")
    else:
        data_arr = np.array(data)
        dataset = h5_file.create_dataset(name, data_arr.shape, data_arr.dtype)
        if len(data_arr.shape) == 0:
            dataset = data_arr
        else:
            for i in range(data_arr.shape[0]):
                dataset[i] = data_arr[i]

In [ ]:
with h5py.File(cnmf_fpath, "r") as hf:
    A_data = hf["estimates"]["A"]["data"][()]
    A_indptr = hf["estimates"]["A"]["indptr"][()]
    A_indices = hf["estimates"]["A"]["indices"][()]
    A_shape = hf["estimates"]["A"]["shape"][()]
# test that spatial to be exported is same as cnmf object's spatial components
spatial2 = scipy.sparse.csc.csc_matrix((A_data, A_indices, A_indptr), shape=A_shape)
assert (spatial2.todense() == cnmf.estimates.A.todense()).all()

In [ ]:
# TODO: test moco intervals, flags, cnmf flags saved format
with h5py.File(whole_traces_h5_fpath, 'w') as hf:
    print("Adding uuid")
    hf.attrs["uuid"] = session_uuid
    print("Adding MoCo intervals")
    append_dataset(hf, "moco_intervals", moco_intervals)
    print("Adding MoCo flags")
    append_dataset(hf, "moco_flags", moco_flags)
    print("Adding cnmf_intervals")
    append_dataset(hf, "cnmf_intervals", cnmf_intervals)
    print("Adding cnmf_flags")
    append_dataset(hf, "cnmf_flags", cnmf_flags)
    print("Adding begin_end_frames")
    append_dataset(hf, "begin_end_frames", moco_data_dict["begin_end_frames"])
    print("Saving spatial...")
    spatial_group = hf.create_group("spatial")  # save "spatial" as-is takes up more space but easier to access
    spatial_group.create_dataset("data", data=A_data)
    spatial_group.create_dataset("indices", data=A_indices)
    spatial_group.create_dataset("indptr", data=A_indptr)
    spatial_group.create_dataset("shape", data=A_shape)
    print("Saving traces...")
    hf.create_dataset("traces", data=traces_complete)
print(f"Saved listed parameters and data in\n\t{whole_traces_h5_fpath}")